In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv(
    "household_power_consumption.txt",
    sep=";",    
    infer_datetime_format=True
)

/tmp/ipykernel_1632/2386425759.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
/tmp/ipykernel_1632/2386425759.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [4]:
df.iloc[:, 2:-1] = df.iloc[:,2:-1].replace('?',np.nan)

In [5]:
df.iloc[:, 2:-1].isnull().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
dtype: int64

In [6]:
df_clean = df

In [7]:
cols = df_clean.columns[2:-1]
df_clean[cols] = df_clean[cols].apply(lambda x: pd.to_numeric(x.str.strip(), errors='coerce'))

In [8]:
df_clean = df.dropna()

In [9]:
df_clean[["Date","Time"]]

,Date,Time
0,16/12/2006,17:24:00
1,16/12/2006,17:25:00
2,16/12/2006,17:26:00
3,16/12/2006,17:27:00
4,16/12/2006,17:28:00
...,...,...
2031611,27/10/2010,13:35:00
2031612,27/10/2010,13:36:00
2031613,27/10/2010,13:37:00
2031614,27/10/2010,13:38:00


In [10]:
df_clean["Day"] = df_clean["Date"].astype(str).str.split("/").str[0].astype(int)
df_clean["Month"] = df_clean["Date"].astype(str).str.split("/").str[1].astype(int)
df_clean["Hour"] = df_clean["Time"].astype(str).str.split(":").str[0].astype(int)

/tmp/ipykernel_1632/914206063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Day"] = df_clean["Date"].astype(str).str.split("/").str[0].astype(int)
/tmp/ipykernel_1632/914206063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Month"] = df_clean["Date"].astype(str).str.split("/").str[1].astype(int)
/tmp/ipykernel_1632/914206063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [11]:
df_clean.drop("Date",inplace=True,axis=1)
df_clean.drop("Time",inplace=True,axis=1)

/tmp/ipykernel_1632/3778227572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop("Date",inplace=True,axis=1)
/tmp/ipykernel_1632/3778227572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop("Time",inplace=True,axis=1)


In [ ]:
X = df_clean.drop("Global_active_power", axis=1)
y = df_clean["Global_active_power"]

In [29]:
from sklearn.linear_model import LinearRegression
import xgboost as xg
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

models = {
    "xgb": xg.XGBRegressor(),
    "LinearRegression": LinearRegression(),

}

results = {}

for name, model in models.items():
    pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    results[name] = r2


In [43]:
best_model = max(results,key=results.get)
best_model = models[best_model]

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("best_model", best_model)
    ])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_pred

array([1.0039011 , 0.37106884, 4.018336  , ..., 2.2870185 , 1.4120381 ,
       0.23398831], dtype=float32)

In [ ]:
import joblib


['best_model.pkl']

In [42]:
model = joblib.load("best_model.pkl")
y_pred = model.predict(X_test)
y_pred

array([1.0039011 , 0.37106884, 4.018336  , ..., 2.2870185 , 1.4120381 ,
       0.23398831], dtype=float32)